In [1]:
import pandas as pd
import numpy as np
import os
import sys

sys.path.append("../..")
from analysis_functions import *

In [2]:
# Загрузка данных
X_stress_components_new = opener("X_stress_components_new", path_import="../../resourses")
X_strain_components_new = opener("X_strain_components_new", path_import="../../resourses")
y_stress_components_new = opener("y_stress_components_new", path_import="../../resourses")
y_strain_components_new = opener("y_strain_components_new", path_import="../../resourses")
# X_stress_components_other = opener('X_stress_components_other')
# X_strain_components_other = opener('X_strain_components_other')
# y_stress_components_other = opener('y_stress_components_other')
# y_strain_components_other = opener('y_strain_components_other')


../../resourses/X_stress_components_new.pkl
../../resourses/X_strain_components_new.pkl
../../resourses/y_stress_components_new.pkl
../../resourses/y_strain_components_new.pkl


In [3]:
component_num = 2

X, y = (X_stress_components_new[component_num], 
        y_stress_components_new[component_num])

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import root_mean_squared_error


# Разделение данных на обучающую, валидационную и тестовую выборки
cur_X_train, cur_X_test, cur_y_train, cur_y_test = split_transform_one_comp_train_test(
    X, y
)

# Обучение модели
model = DecisionTreeRegressor(
    random_state=RANDOM_STATE, max_depth=10, min_samples_split=2
)
model.fit(cur_X_train, cur_y_train)

# Предсказание
cur_y_pred = model.predict(cur_X_test)

# Оценка качества
rmse = root_mean_squared_error(cur_y_test, cur_y_pred)
print(f"RMSE test: {rmse:.2f}")

# На трейне
cur_y_pred_train = model.predict(cur_X_train)
rmse_train = root_mean_squared_error(cur_y_train, cur_y_pred_train)
print(f"RMSE train: {rmse_train:.2f}")

RMSE test: 3.63
RMSE train: 3.41


In [5]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import root_mean_squared_error


# Разделение данных на обучающую, валидационную и тестовую выборки
cur_X_train, cur_X_test, cur_y_train, cur_y_test = split_transform_one_comp_train_test(
    X, y
)

# Обучение модели
model = DecisionTreeRegressor(
    random_state=RANDOM_STATE, max_depth=10, min_samples_split=2
)
model.fit(cur_X_train, cur_y_train)

# Предсказание
cur_y_pred = model.predict(cur_X_test)

# Оценка качества
rmse = root_mean_squared_error(cur_y_test, cur_y_pred)
print(f"RMSE test: {rmse:.2f}")

# На трейне
cur_y_pred_train = model.predict(cur_X_train)
rmse_train = root_mean_squared_error(cur_y_train, cur_y_pred_train)
print(f"RMSE train: {rmse_train:.2f}")

RMSE test: 3.63
RMSE train: 3.41


## Простое обучение

In [6]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import root_mean_squared_error


# Разделение данных на обучающую, валидационную и тестовую выборки
cur_X_train, cur_X_test, cur_y_train, cur_y_test = split_transform_one_comp_train_test(
    X, y
)

# Обучение модели
model = DecisionTreeRegressor(
    random_state=RANDOM_STATE, max_depth=10, min_samples_split=2
)
model.fit(cur_X_train, cur_y_train)

# Предсказание
cur_y_pred = model.predict(cur_X_test)

# Оценка качества
rmse = root_mean_squared_error(cur_y_test, cur_y_pred)
print(f"RMSE test: {rmse:.2f}")

# На трейне
cur_y_pred_train = model.predict(cur_X_train)
rmse_train = root_mean_squared_error(cur_y_train, cur_y_pred_train)
print(f"RMSE train: {rmse_train:.2f}")

RMSE test: 3.63
RMSE train: 3.41


## Добавляю кросс-валидацию

In [10]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import root_mean_squared_error
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score


n_splits = 3
# Preparing datasets
cur_X_test, cur_y_test, val_list_X, val_list_y, train_list_X, train_list_y = (
    split_transform_one_comp_cv(X, y, n_splits=n_splits)
)


def do_optuna_for_tree(X, y, n_trials=100, **kwargs):
    n_splits = kwargs.get("n_splits", 3)

    def optuna_decision_tree_val(trial):
        max_depth = trial.suggest_int("max_depth", 1, 32)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 32)

        params = {
            "max_depth": max_depth,
            "min_samples_split": min_samples_split,
            "random_state": RANDOM_STATE,
        }

        # Fitting and scoring `n_split` times
        errors = np.zeros((n_splits, 9))

        for split_idx in range(n_splits):
            regr = DecisionTreeRegressor(**params)

            cur_X_train = train_list_X[split_idx]
            cur_y_train = train_list_y[split_idx]

            cur_X_val = val_list_X[split_idx]
            cur_y_val = val_list_y[split_idx]

            cur_X_train, cur_y_train = clean_input_array(cur_X_train, cur_y_train)
            cur_X_val, cur_y_val = clean_input_array(cur_X_val, cur_y_val)

            regr.fit(cur_X_train, cur_y_train)

            #######  Validation  ########
            #  Prediction
            cur_prediction = regr.predict(cur_X_val)
            # Scoring
            errors[split_idx] = scorer(cur_y_val, cur_prediction, regr, cur_X_train)

        # Collect validation result
        val_metrics = choose_worst(errors)
        return_value = (
            val_metrics[-1] if pd.notnull(val_metrics[-1]) else +1e6
        )  # для rmse
        return return_value

    # Create a study object to optimize the objective
    study = optuna.create_study(direction="minimize")  # rmse
    study.optimize(optuna_decision_tree_val, n_trials=n_trials, n_jobs=-1)

    # Print the best hyperparameters found by Optuna
    best_params = study.best_params
    best_value = study.best_value
    print("Best Hyperparameters:", best_params)

    return best_params, cur_X_test, cur_y_test, best_value


# Use the function
best_params, cur_X_test, cur_y_test, best_value = do_optuna_for_tree(X, y, n_splits=n_splits)
# Предсказание
cur_y_pred = model.predict(cur_X_test)

# Оценка качества
rmse = root_mean_squared_error(cur_y_test, cur_y_pred)
print(f"RMSE test: {rmse:.2f}")

# На трейне
cur_y_pred_train = model.predict(cur_X_test)
rmse_train = root_mean_squared_error(cur_y_test, cur_y_pred_train)
print(f"RMSE train: {rmse_train:.2f}")


[I 2025-03-02 00:15:16,089] A new study created in memory with name: no-name-06a95016-e16a-41a4-a983-fcf51fb47d89
[I 2025-03-02 00:15:16,170] Trial 4 finished with value: 31.634105073169398 and parameters: {'max_depth': 2, 'min_samples_split': 23}. Best is trial 4 with value: 31.634105073169398.
[I 2025-03-02 00:15:16,185] Trial 2 finished with value: 14.813776915953854 and parameters: {'max_depth': 4, 'min_samples_split': 32}. Best is trial 2 with value: 14.813776915953854.
[I 2025-03-02 00:15:16,234] Trial 1 finished with value: 7.065086709030752 and parameters: {'max_depth': 7, 'min_samples_split': 16}. Best is trial 1 with value: 7.065086709030752.
[I 2025-03-02 00:15:16,257] Trial 5 finished with value: 2.540380896100325 and parameters: {'max_depth': 25, 'min_samples_split': 31}. Best is trial 5 with value: 2.540380896100325.
[I 2025-03-02 00:15:16,296] Trial 7 finished with value: 2.2232613806691317 and parameters: {'max_depth': 13, 'min_samples_split': 10}. Best is trial 7 with 

Best Hyperparameters: {'max_depth': 26, 'min_samples_split': 3}
RMSE test: 3.55
RMSE train: 3.55
